## Librerias 

In [1]:
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import box
from shapely.geometry import Point
import pandas as pd
import numpy as np
import fiona
from shapely.geometry import Polygon

In [2]:
fiona.drvsupport.supported_drivers['libkml'] = 'rw'  # Habilitar soporte para KML
fiona.drvsupport.supported_drivers['LIBKML'] = 'rw'  # Habilitar soporte para KML

## Function to obtain only northern routes and format them

In [3]:
def getGdfN(ruta_archivo,color):
    """"
    Esta función carga un archivo KML, crea un GeoDataFrame y realiza una intersección con un polígono predefinido.
    
    Parámetros:
    ruta_archivo (str): Ruta al archivo KML.
    color (str): Color para el trazo del GeoDataFrame.
    
    Devuelve:
    Df_procesado (GeoDataFrame): GeoDataFrame después de la intersección.
    """
        # Definir las coordenadas de latitud y longitud del poligno que delimita el area de interes
    lat1, lon1 = -3.021520, -79.927455
    lat2, lon2 = -5.589162, -76.995614
    lat4, lon4 = -10.086451, -78.151366
    lat3, lon3 = -9.575868, -76.735243
    lat5, lon5 = -4.711561, -83.303894
    
    # Crear el objeto Polygon
    polygon = Polygon([(lon1, lat1), (lon2, lat2), (lon3, lat3), (lon4, lat4), (lon5, lat5)])
    
    # Imprimir el objeto Polygon
    polys0 = gpd.GeoSeries(polygon) # Creo una geoseries del poligono
    Poly_gdf = gpd.GeoDataFrame({'geometry': polys0}) #creo un geoDf del poligono agregando el nombre de columna 
    df1.crs = "EPSG:4979" #This value depends on your geographic location.
    #### Este poligono en este formato es necesario para poder usar la funcion .overlay, que recive 2 geodf de argumentos ####
    ##############################################################################
    Rutas_gdf = gpd.read_file(ruta_archivo)
    Rutas_gdf = Rutas_gdf[['Name', 'description', 'geometry']] #Columnas que me sirven 
    Rutas_gdf.loc[:,'stroke'] = color #Color uniforme, formato
    Rutas_gdf.loc[:,'stroke-width'] = 2.0 
    Rutas_gdf.loc[:,'stroke-opacity'] = 1.0
    Rutas_gdf = Rutas_gdf[['Name','description','stroke','stroke-width','stroke-opacity','geometry']]
    Rutas_gdf.crs = "EPSG:4979" #asigno el mismo CRS #3D
    # Ahora puedes hacer la superposición
    Df_procesado = gpd.overlay(Rutas_gdf, df1, how='intersection')# Este metodo reemplaza a la interseccion con el poligono (es mas exacto)
    return Df_procesado

## Rutas Existentes y Rutas en Proceso 2023

In [4]:
# Leer el archivo KML directamente todos los datos geograficos
ruta_archivo = 'Data/KML/EXISTENTE PROPIO.kml'
ExsNorte = getGdfN(ruta_archivo,'#ff0000')
# Leer el archivo KML directamente todos los datos geograficos
ruta_archivo1 = 'Data/KML/EN PROCESO 2023.kml'
En_Proceso2023N = getGdfN(ruta_archivo1,'#990000')

## Añadir Poligonos 

In [8]:
# Leer el archivo GeoJSON
Poli_path = 'Data/KML/PoligonosV0.2.kml'
Plig = gpd.read_file(Poli_path)

## Formato para el df de los polignos 

In [10]:
#Asigno colores de lo spoligonos en HEX
Colores = ['#CD6155', '#A569BD', '#5DADE2','#45B39D', '#58D68D','#F5B041', 
           '#04364A', '#DC7633','#99A3A4', '#5D6D7E', '#19C6BE', '#1968C6','#50BD27',] #definir colores
#### I create the columns to satisfy the format##########
Plig.crs = "EPSG:4979" #CRS for #3D geometrys
#Normalizo columnas para el cocat
Plig['stroke-opacity'] = np.nan
Plig['stroke'] = np.nan 
Plig['stroke-width'] = np.nan
Plig['fill'] = np.nan
Plig.loc[:,'stroke-opacity'] = Plig['stroke-opacity'].apply(lambda x: '0.6') #Defino la opacidad
Plig['stroke'] = Colores #Pinto bordes
Plig['fill'] = Colores #Pinto relleno
#Ordeno las columnas
Plig = Plig[['Name','description','stroke','stroke-width','stroke-opacity','fill','geometry']]

# Rutas Conjuntas

In [12]:
Data_u = pd.concat([ExsNorte,En_Proceso2023N])
Data_u.crs

<Geographic 3D CRS: EPSG:4979>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
- h[up]: Ellipsoidal height (metre)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [13]:
Data_App = pd.concat([Data_u, Plig]) #Fuziono todo en un solo mapa
Data_App.reset_index(drop=True)

,Name,description,stroke,stroke-width,stroke-opacity,geometry,fill
0,Proyectado Chacas,1.99 Km,#ff0000,2.0,1.0,"LINESTRING Z (-77.01838 -9.16132 0.00000, -77....",NaN
1,ToRontoy,7.76 Km,#ff0000,2.0,1.0,"LINESTRING Z (-76.97896 -9.15634 0.00000, -76....",NaN
2,None,None,#ff0000,2.0,1.0,"LINESTRING Z (-78.84157 -7.22497 0.00000, -78....",NaN
3,MT HIDRANDINA 1.5 KM,None,#ff0000,2.0,1.0,"LINESTRING Z (-78.56335 -7.27692 0.00000, -78....",NaN
4,MT HIDRANDINA 1.3 KM,None,#ff0000,2.0,1.0,"LINESTRING Z (-78.65051 -7.26036 0.00000, -78....",NaN
...,...,...,...,...,...,...,...
293,Tumbes,None,#99A3A4,NaN,0.6,"POLYGON Z ((-80.79492 -3.72017 0.00000, -80.98...",#99A3A4
294,Chiclayo,None,#5D6D7E,NaN,0.6,"POLYGON Z ((-79.93080 -6.87379 0.00000, -79.77...",#5D6D7E
295,Trujillo,None,#19C6BE,NaN,0.6,"POLYGON Z ((-79.06282 -8.14946 0.00000, -78.85...",#19C6BE
296,Huaraz,None,#1968C6,NaN,0.6,"POLYGON Z ((-77.90299 -8.85158 0.00000, -77.64...",#1968C6


In [14]:
Data_App.explore(color=Data_App['stroke'])

# Calculo de Km de Fibra y Km2 de Poligonos

In [17]:
def get_polygondf_and_Km1(polygon_df ,Base_de_Cuadrilla,route):
    """
    Esta función toma un DataFrame de polígonos, un nombre de cuadrilla y una ruta, 
    y devuelve un GeoDataFrame de las rutas que intersectan con el polígono de la cuadrilla 
    y la longitud total de estas rutas en kilómetros.
    
    Parámetros:
    DFofPolg (GeoDataFrame): DataFrame de polígonos.
    Base_de_Cuadrilla (str): Nombre de la cuadrilla.
    RutaClaro (GeoDataFrame): GeoDataFrame de las rutas.
    
    Devuelve:
    place_geodf (GeoDataFrame): GeoDataFrame de las rutas que intersectan con el polígono de la cuadrilla.
    longitud_total_km (float): Longitud total de las rutas en kilómetros.
    """
    Crs = 'EPSG:32717' # para el emisferio sur, coordenadas planas
    plg_df = DFofPolg.loc[(DFofPolg['Name'] == Base_de_Cuadrilla)].geometry #filas que coincidan con la cuadrilla 
    place_geodf = gpd.overlay(RutaClaro, gpd.GeoDataFrame(plg_df), how='intersection') # Intersecto el poligno con las rutas 
    rutas_proyectadas = place_geodf.to_crs(Crs) 
    place_geodf.loc[:,'Longitud'] = rutas_proyectadas.length  #Creo columna con las longitudes de  las rutas 
    longitud_total_km = place_geodf['Longitud'].sum()/1000 
    return place_geodf, longitud_total_km 
Places = Plig.Name.tolist()
dict_de_geodf = {} # En este diccionrio estan almacenado todas las rutas de cada cuadrilla 
km_fo = []
for place in Places: 
    place_geodf,LFO = get_polygondf_and_Km1(Plig,place,ExsNorte) 
    km_fo.append(LFO)
    dict_de_geodf[place] = place_geodf
    print("Hay" ,LFO, "Kilometros de FibraO en", place)

Hay 210.61240848831912 Kilometros de FibraO en NuevoChimbote
Hay 237.25105987517344 Kilometros de FibraO en PredoRuiz
Hay 469.4142297752381 Kilometros de FibraO en Jaen
Hay 125.27562153379212 Kilometros de FibraO en Talara
Hay 171.22913281498825 Kilometros de FibraO en Huamachuco
Hay 283.60240370729196 Kilometros de FibraO en Cajamarca
Hay 256.1818459448176 Kilometros de FibraO en Huancabamba
Hay 490.364895160399 Kilometros de FibraO en Piura
Hay 132.46418515620513 Kilometros de FibraO en Tumbes
Hay 305.69364758782433 Kilometros de FibraO en Chiclayo
Hay 239.15724538212197 Kilometros de FibraO en Trujillo
Hay 360.37215047678006 Kilometros de FibraO en Huaraz
Hay 102.25954071474018 Kilometros de FibraO en Aragostay


In [18]:
##### Creo un csv para cada base de cuadrilla ######
for key, df in dict_de_geodf.items():
    ruta = 'DFxBase_Cuadrila/'   
    df.to_csv(f'{ruta}{key}.csv')

In [19]:
a= []
def CalcKm2(df,Base_de_Cuadrilla):  
    df0 = df.to_crs(epsg=32718) #este CRS demuestra ser el mas exacto hasta ahora 19/10/23 para areas
    area = df0.loc[(df0['Name'] == Base_de_Cuadrilla)].geometry.area
    for flt in area:
        area_km2 = flt/1000000
        a.append(area_km2)
    return area_km2, a

In [20]:
for place in Places:
    place_km2, ary = CalcKm2(Plig, place)
    print("El poligono cubre" ,place_km2, "Kilometros2 en", place)

El poligono cubre 8965.448526756638 Kilometros2 en NuevoChimbote
El poligono cubre 6341.3995876094 Kilometros2 en PredoRuiz
El poligono cubre 9056.977104990896 Kilometros2 en Jaen
El poligono cubre 1360.253602200892 Kilometros2 en Talara
El poligono cubre 2366.4841207002833 Kilometros2 en Huamachuco
El poligono cubre 10028.944233167236 Kilometros2 en Cajamarca
El poligono cubre 7227.9571527483 Kilometros2 en Huancabamba
El poligono cubre 12777.309208882089 Kilometros2 en Piura
El poligono cubre 1197.548808395513 Kilometros2 en Tumbes
El poligono cubre 9126.412689163195 Kilometros2 en Chiclayo
El poligono cubre 7864.6546544568255 Kilometros2 en Trujillo
El poligono cubre 10502.99430082083 Kilometros2 en Huaraz
El poligono cubre 1132.062872455537 Kilometros2 en Aragostay


In [21]:
df = pd.DataFrame({'Base de Cuadrilla' :Places, 'Km de FibraO': km_fo, 'Km2 del Poligono':ary})
df

,Base de Cuadrilla,Km de FibraO,Km2 del Poligono
0,NuevoChimbote,210.612408,8965.448527
1,PredoRuiz,237.251060,6341.399588
2,Jaen,469.414230,9056.977105
3,Talara,125.275622,1360.253602
4,Huamachuco,171.229133,2366.484121
5,Cajamarca,283.602404,10028.944233
6,Huancabamba,256.181846,7227.957153
7,Piura,490.364895,12777.309209
8,Tumbes,132.464185,1197.548808
9,Chiclayo,305.693648,9126.412689


# Preprocesamiento 4 PowerBi

In [22]:
Maping4BC = {'NuevoChimbote' : 'NUEVOCHIMBOTE,ANCASH', ## Doy el formato que estoy usando en power BI ##
             'PredoRuiz' : 'PEDRORUIZ,AMAZONAS',
             'Jaen' : 'JAEN,CAJAMARCA',
             'Talara' : 'TALARA,PIURA',
             'Huamachuco' : 'HUAMACHUCO,LALIBERTAD',
             'Cajamarca' : 'CAJAMARCA,CAJAMARCA',
             'Huancabamba' : 'HUANCABAMBA,PIURA',
             'Piura' : 'PIURA,PIURA',
             'Tumbes' : 'TUMBES,TUMBES',
             'Chiclayo' : 'CHICLAYO,LAMBAYEQUE',
             'Trujillo' : 'TRUJILLO,LALIBERTAD',
             'Huaraz' : 'HUARAZ,ANCASH',
             'Aragostay' : 'ARAGOSTAY,LALIBERTAD'}

In [23]:
df['Base de Cuadrilla'] = df['Base de Cuadrilla'].replace(Maping4BC) #Aplico el mapping

In [24]:
df.to_csv('4BI.csv', index=False)

In [25]:
MapaJson = Data_App.to_json()
with open('Mapa.json', 'w') as file:
    file.write(MapaJson)